#  e-CHURN XGBoost

### Material de referência.
xgboos: https://gist.github.com/pb111/cc341409081dffa5e9eaf60d79562a03
shap: https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/tree_based_models/Basic%20SHAP%20Interaction%20Value%20Example%20in%20XGBoost.html
lime: https://coderzcolumn.com/tutorials/machine-learning/how-to-use-lime-to-understand-sklearn-models-predictions#lime_tabular


### Etapas:
- Carga de bibliotecas e bases
- Preparação de dados
- Roda modelo simples
- Faz validação cruzada
- Pega aprendizados de modelo e variáveis e aplica modelo otimizado

## Carga de bibliotecas e bases

In [1]:
!pip install xgboost
!pip install skimpy
!pip install shap
!pip install openpyxl
!pip install lime
!pip install pandas

  Using cached skimpy-0.0.15-py3-none-any.whl (16 kB)
  Using cached rich-13.8.1-py3-none-any.whl (241 kB)
  Using cached typeguard-4.2.1-py3-none-any.whl (34 kB)
  Using cached pyarrow-16.1.0-cp310-cp310-manylinux_2_28_x86_64.whl (40.8 MB)
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Using cached polars-0.20.31-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (28.8 MB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfull

In [2]:
# Carregar bibliotecas e setar parâmetros
#!pip install xgboost
#!pip install skimpy
#!pip install shap
#!pip install openpyxl
#!pip install lime
#!pip install pandas
#!pip install --upgrade pandas==1.2

import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import cv
import lime #LIME package
import lime.lime_tabular #the type of LIIME analysis we’ll do
from lime import lime_tabular
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
#from skimpy import clean_columns #limpar nomes de coluna
import matplotlib.pyplot as plt #graficos
import re #expressoes regulares
import numpy as np
import seaborn as sns
import shap
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Ler e preparar base
df = pd.read_excel('Base_ECHURN_202408.xlsx')


In [ ]:
df.columns

In [ ]:

print(df.head(1).to_string())

In [ ]:


#df = df[df['cod_empresa']==1]

df2 = df[df['treino']==0]
df[df['treino']==1]


df2.drop (['treino','num_span','ID','agrupado_nivel','macro_especialidade','num_report_total','desc_escolaridade'],axis=1,inplace=True)
df.drop (['treino','num_span','ID','agrupado_nivel','macro_especialidade','num_report_total','desc_escolaridade'],axis=1,inplace=True)


X = df.drop(['status'], axis=1)
y = df['status']

Xt = df2.drop(['status'], axis=1)
yt = df2['status']

## Pre tratamento dos dados

In [ ]:
# Tratamento dos dados

# Quebra variaveis categoricas em multiplas 1 e 0
#sexo de um homem vira sexo_masculino=1 e sexo_feminino=0
X = pd.get_dummies(X)

#conserta nomes das colunas
#X = clean_columns(X)

# Visão das primeiras linhas da tabela
#X.head(5)

print(X.head(2).to_string())

In [ ]:
X.columns


In [ ]:
# Prepara os dados para o modelo

# Converte tabela para data_dmatrix (formato de dado otimizado para o xgboost)
data_dmatrix = xgb.DMatrix(data=X,label=y)

# Quebra os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
# Construção de modelo simples
params = {
            'objective':'binary:logistic',
            'learning_rate': 1.0,   # 0-1 prevenir overfitting
            'max_depth': 4,         # Profundidade maxima por arvore
            'subsample':1,          # % de amostras por arvore
            'colsample_bytree':0.3, # % de cats por arvore
            'n_estimators':100,     # num de arvores
            'seed':42               # manter reprodutibilidade
        }

# instantiate the classifier 
xgb_clf = XGBClassifier(**params)

# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

In [ ]:
# Faz previsao no conjunto de teste
y_pred = xgb_clf.predict(X_test)

# Mede score de acuracia
from sklearn.metrics import accuracy_score
print('XGBoost model accuracy score: {0:0.2f}'. format(
    accuracy_score(y_test, y_pred)))

## Fazer validação cruzada

In [ ]:
params = {"objective":"binary:logistic",
          'colsample_bytree': 0.3,
          'eta':0.05,
          'learning_rate': 1.0,
          'max_depth': 4, 
          'alpha': 10
         }

xgb_cv = cv(dtrain=data_dmatrix, params=params, nfold=3,
            num_boost_round=1000, early_stopping_rounds=100,
            metrics="error", as_pandas=True, seed=42)

#xgb_cv.head(5)

print(xgb_cv.head(1).to_string())

In [ ]:
fig, ax = plt.subplots()
ax.scatter(xgb_cv.index.values.astype(int),
           xgb_cv['test-error-mean'])
#ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Rounds')
ax.set_ylabel('Erro')
ax.set_title('Erro_por_Rounds')
plt.show();

In [ ]:
xgb.plot_importance(xgb_clf)
plt.rcParams['figure.figsize'] = [6, 4]
plt.show()

## Coleta aprendizados e aplica um modelo bom

In [ ]:
# Prepara os dados para o modelo
X_original = X
lista_var = ['clas_faixa_neon','level','tempo_nivel',
                'tempo_cargo','num_tempo_casa','Key_Talent',
                'num_tempo_casa','flag_socio','num_estado_civil', 'media_BH','num_nivel'
            ]
X = X_original[lista_var]
# Converte tabela para data_dmatrix (formato de dado otimizado para o xgboost)
data_dmatrix = xgb.DMatrix(data=X,label=y)
# Quebra os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
# Construção de modelo simples
params = {
            'objective':'binary:logistic',
            'learning_rate': 1.0,   # Ajuda a prevenir overfitting
            'max_depth': 4,         # Profundidade maxima por arvore
            'subsample':1,          # % de amostras por arvore
            'colsample_bytree':0.3, # % de cats por arvore
            'n_estimators':100,      # num de arvores
            'seed':42               # manter reprodutibilidade
        }
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)
# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)
# Faz previsao no conjunto de teste
y_pred = xgb_clf.predict(X_test)

# Mede score de acuracia
from sklearn.metrics import accuracy_score
print('XGBoost model accuracy score: {0:0.2f}'. format(
    accuracy_score(y_test, y_pred)))


In [ ]:
from sklearn.metrics import confusion_matrix

#print(confusion_matrix(y_test, xgb_clf.predict(X_test)))

cm = confusion_matrix(y_test, xgb_clf.predict(X_test))
  
TN = cm[0][0]
TP = cm[1][1]
FN = cm[1][0]
FP = cm[0][1]
  
print(cm)
print('Model Testing Accuracy = "{0:0.2f}!"'.format(  (TP + TN) / (TP + TN + FN + FP)))
print()

In [ ]:
print(classification_report(y_test, xgb_clf.predict(X_test)))

In [ ]:
# make sure the SHAP values add up to marginal predictions
pred = xgb_clf.predict(X, output_margin=True)
explainer = shap.TreeExplainer(xgb_clf)
shap_values = explainer.shap_values(X)
np.abs(shap_values.sum(1) + explainer.expected_value - pred).max()

In [ ]:
shap.summary_plot(shap_values, X, plot_type="bar")


In [ ]:
shap.summary_plot(shap_values, X)

In [ ]:
# Monta tabela para comparaçã ode resultados
status = pd.DataFrame(y_test.reset_index(drop=True))
status.rename({'Status': 'status'}, axis=1, inplace=True)
pred = pd.DataFrame(y_pred.tolist(), columns =['pred'])
status['pred'] = pred

status.to_excel(r'status.xlsx', index = False)

#status#.T

In [ ]:
# if a feature has 10 or less unique values then treat it as categorical
categorical_features = np.argwhere(np.array([len(set(X_train.values[:,x]))
for x in range(X_train.values.shape[1])]) <= 10).flatten()
# LIME has one explainer for all models
explainer = lime.lime_tabular.LimeTabularExplainer(
X_train.values,
feature_names=X_train.columns.values.tolist(),
class_names=['Status'],
categorical_features=categorical_features,
verbose=True, mode='regression')

In [ ]:
expXGB = explainer.explain_instance(
X_test.values[2],
xgb_clf.predict, num_features=9)
expXGB.show_in_notebook(show_table=True)